<a href="https://colab.research.google.com/github/dougFE/25Fall-UCLA-ECE117-project2/blob/main/ece117_assignment3_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 117 Assignment 3: Part 1
Training an MNIST model. Goal is to achieve 90+% accuracy.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import tqdm

import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.MNIST("./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST("./data", train=False, download=True, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
# This is provided as a baseline model but feel free to adjust this.

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model = CNN().to(device)

i_max = # choose a value here

criterion = # choose a value here
optimizer = # choose a value here

In [ ]:
@torch.no_grad()
def get_accuracy(model, data_loader, device):
    correct = 0
    total = 0

    for inputs, labels in data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=1)

        total += labels.shape[0]
        correct += int((predicted == labels).sum())

    return correct / total

In [ ]:
progress = tqdm.tqdm(total=i_max, desc="Training")

i = 0
while i < i_max:
    for inputs, labels in train_loader:
        # ============ Forward Pass ==========
        # model's training forward pass

        # move the images and labels to the GPU

        # predict the output of the model

        # calculate the loss with respect to the output


        # ============= Backwards Pass ===========
        # zero the optimizer's gradient

        # perform backpropagation on the loss function

        # call .step() on the optimizer

        i += 1
        progress.update(1)

        if i % 100 == 0:
            train_acc = get_accuracy(model, train_loader, device)
            test_acc = get_accuracy(model, test_loader, device)
            progress.write(f"Iter {i} Train Acc {train_acc:.4f} Test Acc {test_acc:.4f}")

        if i >= i_max:
            break

torch.save(model.state_dict(), "./model.pth")

In [ ]:
model.load_state_dict(torch.load("./model.pth"))

model.eval()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device)
        label = label.to(device)

        pred = model(image)
        _, pred = torch.max(pred, dim=1)

        total += label.shape[0]
        correct += int((pred == label).sum())

    print(f"Accuracy: {correct / total * 100:.2f}%")